In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('./parsed data/table.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6158 entries, 0 to 6157
Data columns (total 8 columns):
multipart_count                         6158 non-null int64
is_spam                                 6158 non-null bool
frm_fuzz_set_ratio                      6158 non-null float64
payload_dale_chall_readability_score    6158 non-null float64
payload_coleman_liau_index              6158 non-null float64
organization                            6158 non-null bool
payload_smog_index                      6158 non-null float64
cc                                      6158 non-null bool
dtypes: bool(3), float64(4), int64(1)
memory usage: 258.7 KB


In [2]:
#normalize data
df['frm_fuzz_set_ratio']=df['frm_fuzz_set_ratio']/np.max(df['frm_fuzz_set_ratio'])
df['payload_dale_chall_readability_score']=df['payload_dale_chall_readability_score']/np.max(df['payload_dale_chall_readability_score'])
df['payload_smog_index']=df['payload_smog_index']/np.max(df['payload_smog_index'])
df['payload_coleman_liau_index']=df['payload_coleman_liau_index']/np.max(df['payload_coleman_liau_index'])
df.head()

,multipart_count,is_spam,frm_fuzz_set_ratio,payload_dale_chall_readability_score,payload_coleman_liau_index,organization,payload_smog_index,cc
0,0,False,0.93,0.277884,0.006038,False,0.150943,True
1,0,False,1.00,0.274870,0.009278,False,0.176101,True
2,0,False,1.00,0.262812,0.004707,False,0.108491,True
3,0,False,1.00,0.278981,0.006193,False,0.163522,True
4,0,False,0.92,0.288572,0.010382,False,0.185535,True


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

if 'is_spam' in df:
    y = df['is_spam'].values # get the labels we want
    del df['is_spam'] # get rid of the class label
    X = df.values# use everything else to predict!

lr_sk = LogisticRegression() # all params default

%time lr_sk.fit(X,y) # no need to add bias term, sklearn does it internally!!
print(lr_sk.coef_)
yhat = lr_sk.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

CPU times: user 31.4 ms, sys: 1.03 ms, total: 32.4 ms
Wall time: 32.4 ms
[[ 0.64941733 -4.29188146  2.0108381   3.32107707 -5.41494561  5.8730554
  -6.13985752]]
Accuracy of:  0.908087041247


In [4]:
#Training/testing 
from sklearn.cross_validation import ShuffleSplit
num_cv_iterations = 8
num_instances = len(y)
cv_object = ShuffleSplit(n=num_instances,
                         n_iter=num_cv_iterations,
                         test_size  = .1)
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None)
# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
for iter_num, (train_indices, test_indices) in enumerate(cv_object):
    lr_clf.fit(X[train_indices],y[train_indices])  # train object
    y_hat = lr_clf.predict(X[test_indices]) # get test set precitions

    # print the accuracy and confusion matrix 
    print("====Iteration",iter_num," ====")
    print("accuracy", mt.accuracy_score(y[test_indices],y_hat)) 
    print("confusion matrix\n",mt.confusion_matrix(y[test_indices],y_hat))

====Iteration 0  ====
accuracy 0.909090909091
confusion matrix
 [[421   4]
 [ 52 139]]
====Iteration 1  ====
accuracy 0.910714285714
confusion matrix
 [[389   1]
 [ 54 172]]
====Iteration 2  ====
accuracy 0.896103896104
confusion matrix
 [[394   4]
 [ 60 158]]
====Iteration 3  ====
accuracy 0.887987012987
confusion matrix
 [[387   2]
 [ 67 160]]
====Iteration 4  ====
accuracy 0.923701298701
confusion matrix
 [[403   5]
 [ 42 166]]
====Iteration 5  ====
accuracy 0.918831168831
confusion matrix
 [[399   2]
 [ 48 167]]
====Iteration 6  ====
accuracy 0.913961038961
confusion matrix
 [[390   5]
 [ 48 173]]
====Iteration 7  ====
accuracy 0.912337662338
confusion matrix
 [[412   4]
 [ 50 150]]


In [8]:
from sklearn.preprocessing import StandardScaler
# we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)

from sklearn.pipeline import Pipeline
import plotly
# you can apply the StandardScaler function inside of the cross-validation loop 
#  but this requires the use of PipeLines in scikit. 
#  A pipeline can apply feature pre-processing and data fitting in one compact notation
#  Here is an example!

std_scl = StandardScaler()
lr_clf = LogisticRegression(penalty='l2', C=0.05) 

# create the pipline
piped_object = Pipeline([('scale', std_scl),  # do this
                         ('logit_model', lr_clf)]) # and then do this

weights = []
# run the pipline cross validated
for iter_num, (train_indices, test_indices) in enumerate(cv_object):
    piped_object.fit(X[train_indices],y[train_indices])  # train object
    # it is a little odd getting trained objects from a  pipeline:
    weights.append(piped_object.named_steps['logit_model'].coef_[0])
    

weights = np.array(weights)

/Users/arslanmemon/miniconda2/envs/MLEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning:

Data with input dtype object was converted to float64 by StandardScaler.

/Users/arslanmemon/miniconda2/envs/MLEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning:

Data with input dtype object was converted to float64 by StandardScaler.

/Users/arslanmemon/miniconda2/envs/MLEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning:

Data with input dtype object was converted to float64 by StandardScaler.

/Users/arslanmemon/miniconda2/envs/MLEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning:

Data with input dtype object was converted to float64 by StandardScaler.

/Users/arslanmemon/miniconda2/envs/MLEnv/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning:

Data with input dtype object was converted to float64 by StandardScaler.



In [9]:
error_y=dict(
            type='data',
            array=np.std(weights,axis=0),
            visible=True
        )

graph1 = {'x': df.columns,
          'y': np.mean(weights,axis=0),
    'error_y':error_y,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)

PlotlyError: Plotly Offline mode has not been initialized in this notebook. Run: 

import plotly
plotly.offline.init_notebook_mode() # run at the start of every ipython notebook